# Recovery Data Partnership - SafeGraph user guide

### 1. Install Dependencies
uncomment the line below and run it

In [ ]:
 # ! pip3 install s3fs pandas

In [ ]:
import pandas as pd
import s3fs

We will be using `s3fs` to access our files in aws s3. Authentication is handled as below:

In [ ]:
s3 = s3fs.S3FileSystem(
      key='<YOUR KEY HERE>',
      secret='<YOUR SECRET HERE>', 
      client_kwargs={
          'endpoint_url': 'https://s3.amazonaws.com', 
          'region_name':'us-east-1'
      }
    )

### 2. Listing available files

`s3.ls` will allow you to list directories given s3 path. For this project specifically, we will use `recovery-data-partnership/output` folder as the root folder for delivering all output files


In [ ]:
s3.ls('recovery-data-partnership/output/production/')

['recovery-data-partnership/output/production/',
 'recovery-data-partnership/output/production/home_panel_summary',
 'recovery-data-partnership/output/production/lookups',
 'recovery-data-partnership/output/production/social_distancing']

Under the `output/production/` folder, you will find data products by category. All social distancing related data products will be stored under `social_distancing`, and all point of interest and patterns datasets will be stored under `poi`. 

In [ ]:
s3.ls('recovery-data-partnership/output/production/social_distancing/')

['recovery-data-partnership/output/production/social_distancing/device_counts_by_county.csv.zip',
 'recovery-data-partnership/output/production/social_distancing/device_counts_by_state.csv.zip',
 'recovery-data-partnership/output/production/social_distancing/weekly_county_trips',
 'recovery-data-partnership/output/production/social_distancing/weekly_state_trips']

Because of the large size of the datasets, we paritioned each output table by year and quarter.

In [ ]:
s3.ls('recovery-data-partnership/output/production/social_distancing/weekly_state_trips/')

['recovery-data-partnership/output/production/social_distancing/weekly_state_trips/weekly_state_trips_2019Q1.csv.zip',
 'recovery-data-partnership/output/production/social_distancing/weekly_state_trips/weekly_state_trips_2019Q2.csv.zip',
 'recovery-data-partnership/output/production/social_distancing/weekly_state_trips/weekly_state_trips_2019Q3.csv.zip',
 'recovery-data-partnership/output/production/social_distancing/weekly_state_trips/weekly_state_trips_2019Q4.csv.zip',
 'recovery-data-partnership/output/production/social_distancing/weekly_state_trips/weekly_state_trips_2020Q1.csv.zip',
 'recovery-data-partnership/output/production/social_distancing/weekly_state_trips/weekly_state_trips_2020Q2.csv.zip',
 'recovery-data-partnership/output/production/social_distancing/weekly_state_trips/weekly_state_trips_2020Q3.csv.zip',
 'recovery-data-partnership/output/production/social_distancing/weekly_state_trips/weekly_state_trips_2020Q4.csv.zip']

### 3. Reading files to dataframe and combine them together

You can easily write a loop to loop through all the year quarter partitions of the datasets and use `pd.concat` to concatenate paritions into one big table.

In [ ]:
dfs = []
for dataset in s3.ls('recovery-data-partnership/output/production/social_distancing/weekly_state_trips/'):
  df = pd.read_csv(s3.open(dataset, mode='rb'), compression='zip')
  dfs.append(df)
  del df

In [ ]:
weekly_state_trips = pd.concat(dfs)

> Note that because we are paritioning by quarter, that means the last week of a quarter can cross quarters and show up in two paritions. To avoid confusion, we recommend doing a group by by week and the geographic boundry.

In [ ]:
weekly_state_trips = weekly_state_trips.groupby(['year_week', 'origin', 'destination']).sum().reset_index()

In [ ]:
weekly_state_trips.head()

,year_week,origin,destination,weekday_trips,weekend_trips,all_trips
0,2019-01,01,NYC,1655.0,894.0,2549
1,2019-01,02,NYC,99.0,35.0,134
2,2019-01,04,NYC,1488.0,708.0,2196
3,2019-01,05,NYC,733.0,505.0,1238
4,2019-01,06,NYC,14045.0,6724.0,20769


### 4. We also prepared look up tables

In [ ]:
s3.ls('recovery-data-partnership/output/production/lookups/')

['recovery-data-partnership/output/production/lookups/',
 'recovery-data-partnership/output/production/lookups/fips_to_county.csv',
 'recovery-data-partnership/output/production/lookups/fips_to_state.csv',
 'recovery-data-partnership/output/production/lookups/naics_sector.csv',
 'recovery-data-partnership/output/production/lookups/naics_subsector.csv',
 'recovery-data-partnership/output/production/lookups/nta_to_boro_county.csv']

The `state_lookup` table will give you the state names

In [ ]:
state_lookup = pd.read_csv(s3.open('recovery-data-partnership/output/production/lookups/fips_to_state.csv', mode='rb'), dtype=str)
state_lookup.head()

,fips,state
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California


In [ ]:
named_weekly_state_trips = pd.merge(weekly_state_trips, state_lookup, how='left', left_on='origin', right_on='fips')
named_weekly_state_trips.head()

,year_week,origin,destination,weekday_trips,weekend_trips,all_trips,fips,state
0,2019-01,01,NYC,1655.0,894.0,2549,01,Alabama
1,2019-01,02,NYC,99.0,35.0,134,02,Alaska
2,2019-01,04,NYC,1488.0,708.0,2196,04,Arizona
3,2019-01,05,NYC,733.0,505.0,1238,05,Arkansas
4,2019-01,06,NYC,14045.0,6724.0,20769,06,California


The device count table for states contains the number of devices that's "home" is in each state. Because the state trips table concerns movement in and out of NYC, the device counts for NY do not include the 5 boroughs. These counts are instead stored as state = 'NYC'. Also note that the devices counts are only associated with the origin location. 


In [ ]:
state_devices = pd.read_csv(s3.open('recovery-data-partnership/output/production/social_distancing/device_counts_by_state.csv.zip', mode='rb'),  compression='zip')

In [ ]:
state_devices.sort_values(by=['year_week', 'state']).head()

,date,year_week,state,device_count,completely_home_device_count
2275,2019-01-01,2019-01,01,411008,158077
5669,2019-01-02,2019-01,01,427789,133183
9880,2019-01-06,2019-01,01,66,28
14304,2019-01-01,2019-01,01,57,27
16911,2019-01-03,2019-01,01,429289,134135


In [ ]:
state_devices = state_devices.groupby(['year_week',	'state']).sum().reset_index()

In [ ]:
state_devices.head()

,year_week,state,device_count,completely_home_device_count
0,2019-01,01,2496438,817154
1,2019-01,02,229101,95159
2,2019-01,04,2683123,1114945
3,2019-01,05,1398385,473440
4,2019-01,06,13137166,5340718


In [ ]:
weekly_state_trips_with_devices = pd.merge(
    weekly_state_trips, state_devices, 
    how='left', 
    left_on=['year_week', 'origin'],
    right_on=['year_week', 'state'])

In [ ]:
weekly_state_trips_with_devices.shape

(11029, 9)

In [ ]:
weekly_state_trips_with_devices.head()

,year_week,origin,destination,weekday_trips,weekend_trips,all_trips,state,device_count,completely_home_device_count
0,2019-01,01,NYC,1655.0,894.0,2549,01,2496438,817154
1,2019-01,02,NYC,99.0,35.0,134,02,229101,95159
2,2019-01,04,NYC,1488.0,708.0,2196,04,2683123,1114945
3,2019-01,05,NYC,733.0,505.0,1238,05,1398385,473440
4,2019-01,06,NYC,14045.0,6724.0,20769,06,13137166,5340718
